# Implementing Faster R-CNN

The objective of this activity is to implement the main parts of the Faster R-CNN algorithm.

We will:
* understand intuitively how the parts of the algorithm are working and how they fit together.
* implement all the stages required for **inference** one by one, using Python, leveraging numpy and TensorFlow.
* use **existing weights** of a Faster R-CNN model that has been trained on the [COCO Dataset](http://cocodataset.org/), to guide and facilitate the process.

We will **NOT**:
* implement any training code whatsoever. We won't code any loss function or deal with ground truth boxes.
* train the model, as we already have weights for you that work.

We've tried to keep code in the notebooks to a minimum, mainly data manipulation and visualization, to make it easy enough to follow. All accompanying code is under the `workshop` Python package.

After some introductory code, the notebook will continue as follows:
* Playing with a **pre-trained ResNet** to obtain features out of an image.
* Generate regions of interest by implementing the **Region Proposal Network** detailed in [1].
* Prepare this regions to be fed to the second stage, by applying **RoI pooling**.
* Classify and refine said regions by passing them through an **R-CNN**, as detailed in [2].

We'll present you with stubs for the different functions required and your task will be to fill them in.

* [1] Ren, Shaoqing, et al. *Faster R-CNN: Towards real-time object detection with region proposal networks.*
* [2] Girshick, Ross. *Fast R-CNN.*

---
# The basics
We'll start with some imports.

The local imports are under the `workshop` package, which you should have installed using `pip install -e workshop/` in the environment you're running your notebook on.

Within `workshop` we have some modules:
* `vis`: various visualization utilities to draw bounding boxes, sliders, etc.
* `image`: utilities for reading images and loading them into PIL (the imaging library).
* `resnet`: the implementation for the base network we're going to use (more on this shortly).
* `faster`: utilities and parts we won't be implementing but provide for completeness' sake.

Let's test some things to make sure everything is up and ready to go.

Start by running the following in your terminal, and then test the rest of the imports:
```bash
 $ jupyter nbextension enable --py widgetsnbextension
 ```

In [ ]:
from ipywidgets import interact, Checkbox, FloatSlider, Layout

import json
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
import tensorflow.contrib.eager as tfe

from PIL import Image


# Try to enable TF eager execution, or do nothing if running again.
try:
    tf.enable_eager_execution()
except ValueError:
    # Already executed.
    pass


# Local imports.
from workshop.faster import (
    clip_boxes, rcnn_proposals, run_base_network, run_resnet_tail,
    generate_anchors_reference, sort_anchors
)
from workshop.image import open_all_images, open_image, to_image
from workshop.vis import (
    add_rectangle, draw_bboxes, draw_bboxes_with_labels, image_grid,
    pager, vis_anchors
)

# Notebook-specific settings.
%matplotlib inline

We'll now load some images to play with and display them below. Change which image is passed to the `to_image` function to see it in full size.

In [ ]:
images = open_all_images('images/')

axes = image_grid(len(images))
for ax, (name, image) in zip(axes, images.items()):
    ax.imshow(np.squeeze(image))
    ax.set_title(name)

plt.subplots_adjust(wspace=.01)
plt.show()

image = images['woman']

# `to_image` turns a `numpy.ndarray` into a PIL image, so it's displayed by the notebook.
to_image(image)

---
# The base network: ResNet


The basis for the Faster R-CNN algorithm is to leverage a pre-trained classifier network to extract feature maps (also called *activation maps*) from the image. For this implementation, we'll be using the popular ResNet 101 [3].

We provide the implementation itself (which you can see in the `workshop.resnet` module), as well as a checkpoint with the pre-trained weights (in the `checkpoint/` directory).

---

### Aside

The ResNet architecture consists of four stacked **blocks**, after which a fully-connected layer is attached. As is expected of CNNs, these blocks detect features from most simple to most complex. For this part of the algorithm, we're using the output of the **block 3**, so we get somewhat generic features. The intuition is that, if we go all the way and use block 4, we might have things that are too specific to the dataset used to pre-train the ResNet (the Imagenet dataset) and thus not as desirable for a network that wants to identify generic objects. 

---

Run the base network on different images, in order to see how the different activation maps behave. **Can you notice any particular features being detected in the activation maps?**

* [3] He, Kaiming, et al. *Deep residual learning for image recognition.*

In [ ]:
with tfe.restore_variables_on_create('checkpoint/fasterrcnn'):
    feature_map = run_base_network(image)

In [ ]:
@interact(page=pager(1024, 20, 'Feature map'))
def display_feature_maps(page):
    axes = image_grid(20)
    for idx, ax in enumerate(axes):
        if page * 20 + idx >= 1024:
            break
        ax.imshow(
            feature_map.numpy()[0, :, :, page * 20 + idx],
            cmap='gray', aspect='auto'
        )

    plt.subplots_adjust(wspace=.01, hspace=.01)
    plt.show()

## Learn: understand what patterns activate particular filters.

Let's overlay the feature maps into the images themselves, so we can take a more detailed look into what pattern makes the ResNet react.

See, for example:
* Feature map 171 in `woman`.
* Feature maps 19, 22 in `cats`.
* Feature map 34, 64 in `bicycles`.
* Feature map 253 in `kids`.

In [ ]:
@interact(idx=pager(1024, 1, 'Feature map index'))
def overlay_feature_map(idx):
    # Normalize the feature map so we get the whole range of colors.
    fm = (
        feature_map.numpy()[0, :, :, idx]
        / feature_map.numpy()[0, :, :, idx].max()
        * 255
    ).astype(np.uint8)
    
    # Resize the feature map without interpolation.
    fm_image = Image.fromarray(fm, mode='L').convert('RGBA')
    fm_image = fm_image.resize(image.shape[1:3][::-1], resample=Image.NEAREST)
    
    # Add some alpha to overlay it over the image.
    fm_image.putalpha(200)
    
    base_image = to_image(image)
    base_image.paste(fm_image, (0, 0), fm_image)
    
    return base_image

This section didn't require any implementation at all, but get ready, because we're about to. The main idea here was illustrating what we mean when we say that the later layers of a classification network are **feature detectors**, reacting to particular patterns in an image.

What would you do if you were to use this information to detect objects? How could you leverage the fact that we can say "there's a cat ear here!"? We're going to explore these questions in the following sections.

For now, back to the slides!

---
# Finding stuff with the Region Proposal Network

Having gone through the theory, we'll now turn our attention to implementing a **Region Proposal Network**. The idea, as we've seen, is to use the feature maps provided by the ResNet to find out **where** there might be an object located.

This is where **anchors** come into play. We'll take a grid of points over the image and consider several anchors (also called "reference boxes" sometimes) for each of them (15 in this case). The RPN layers themselves will then predict whether there's an object in each of these 15 boxes **and** how much we need to resize them to better fit it.

The tasks we have ahead of us are, thus:
* Get the **coordinates** $(x_{min}, y_{min}, x_{max}, y_{max})$ for each of the anchors. There are $15$ anchors and the centers will be separated by approximately $16$ pixels, so we're talking about several thousand of coordinates.
* Find out how to do the special **encoding** and **decoding** of coordinates described in the Faster R-CNN paper, so the RPN can predict locations in the image.
* Build the **convolutional layers** comprising the RPN and run them through different images.
* **Translate the predictions** of the RPN layer into usable proposals.

### Note: coordinate conventions
Except in specific cases, we'll be using the convention $(x_{min}, y_{min}, x_{max}, y_{max})$ to denote a bounding box, were $(x_{min}, y_{min})$ corresponds to the top-left point and $(x_{max}, y_{max})$ the bottom right.

As usual with image processing, the origin of the coordinate system, $(0, 0)$, is on the top-left of the image.

## Generating anchors

We'll get the anchor's coordinates in two steps. First, we will use a function called `generate_anchors_reference` which, given the anchors' settings (i.e. size, aspect ratio, scales), returns an array with the coordinates for the boxes (in pixel space) assuming they're centered around (0, 0). This will give us, effectively, a $(15, 4)$ array.

There are three settings for the anchors:

* `base_size`: **side length for a square anchor**, in pixels (e.g. 256). Increasing it makes the anchor cover more area of the image.
* `scales`: **scale factors** to consider taking `base_size` as reference. For instance, a scale of `2` will make the effective size `512` if base size was `256`.
* `aspect_ratios`: **aspect ratios** of the anchors, expressed as the value of `height / width`. Note that *changing the aspect ratio doesn't change the area the anchor covers*. An aspect ratio of `2` means that, for the area covered by a square anchor of of `base_size`, we should get a rectangle of twice the height than width.

Let's see how this looks like using a single aspect ratio and 3 scales:

In [ ]:
anchors_ref = generate_anchors_reference(
    256,  # Base size.
    [1],  # Aspect ratios.
    [0.5, 1, 2],  # Scales.
)

vis_anchors(anchors_ref)

# Remember this is just a numpy array of shape
#     (total_aspect_ratios * total_scales, 4)
# with the corner points of the reference anchors using the
# convention (x_min, y_min, x_max, y_max).
print('As a numpy array:')
print(anchors_ref)

Let's now change the aspect ratio, but keep the same scales:

In [ ]:
anchors_ref = generate_anchors_reference(
    256,  # Base size.
    [0.5],  # Aspect ratios.
    [0.5, 1, 2],  # Scales.
)

vis_anchors(anchors_ref)

print('As a numpy array:')
print(anchors_ref)

As we are now using $0.5$ as aspect ratio, it means the width/height relation for each anchor should be equal to that (so the rectangles are elongated).

Now, let's try using a single scale, but varying the aspect ratios:

In [ ]:
anchors_ref = generate_anchors_reference(
    256,  # Base size.
    [0.5, 1, 2],  # Aspect ratios.
    [2],  # Scales.
)

vis_anchors(anchors_ref)

print('As a numpy array:')
print(anchors_ref)

In this case, every anchor depicted here should have the same area, since they are all the same **scale** and only vary in their **aspect ratio**.

Finally, let's generate the final set of **15 anchor references** centered around (0,0):

In [ ]:
anchors_ref = generate_anchors_reference(
    256,  # Base size.
    [0.5, 1, 2],  # Aspect ratios.
    [0.125, 0.25, 0.5, 1, 2],  # Scales.
)

vis_anchors(anchors_ref)

print('As a numpy array:')
print(anchors_ref)

For some perspective, let's draw these anchors over **a single arbitrary point** in the image, to see how they match.


Keep in mind that `anchors_ref` is a numpy array containing 15 values, where each one is a rectangle represented as $(x_{min}, y_{min}, x_{max}, y_{max})$.


Since the anchor references are **centered around $(0, 0)$**, it is easy to translate them over any point $P$ by just adding up the coordinates: the anchor $(x_{min}, y_{min}, x_{max}, y_{max})$ at point $P = (x_p, y_p)$ would be specified by the coordinates $(x_{min} + x_p, y_{min} + y_p, x_{max} + x_p, y_{max} + y_p)$.

Let's see how this looks like at point $(400, 270)$:

In [ ]:
point = np.array([400, 270])

# Sum the point on both the *_min and the *_max parts.
anchors_at_point = anchors_ref + np.concatenate([point, point])

_, ax = plt.subplots(1, figsize=(16, 20))
ax.imshow(to_image(image))

# Add a buffer around the image so we see the whole anchor references.
ax.set_xlim([-100, image.shape[2] + 100])
ax.set_ylim([image.shape[1] + 100, -100])

for idx in range(anchors_at_point.shape[0]):
    add_rectangle(ax, anchors_at_point[idx, :])

# Plot the reference point in use.
ax.plot(point[0], point[1], marker='s', color='#dc3912', markersize=3)

plt.show()

As you can see, the larger boxes cover quite a bit of the image, while the smaller ones will be useful for detecting very small objects.


Now, our **first real coding task** (yes!) will be to do the same with the anchor references over each of the **anchor centers** in the image.


Given that we're using a ResNet 101, which has a downsampling factor of 16 (i.e. every point in the feature map --block 3 as we said-- corresponds to a $16\times16$ region of the original image), we'll select the centers **every 16 pixels** in each direction.

For reference, the anchor centers are visualized below.

In [ ]:
# This is actually defined within `run_base_network`, but for visualization
# purposes, we're defining it again.
OUTPUT_STRIDE = 16

# Print the anchor centers in use.
_, ax = plt.subplots(1, figsize=(16, 20))

ax.imshow(to_image(image))
ax.set_xlim([-100, image.shape[2] + 100])
ax.set_ylim([image.shape[1] + 100, -100])

for x in range(0, image.shape[2], OUTPUT_STRIDE):
    for y in range(0, image.shape[1], OUTPUT_STRIDE):
        ax.plot(x, y, marker='s', color='#dc3912', markersize=3)

plt.show()

Let's wrap this part by getting the entire list of anchors for the image. This will be done within the `generate_anchors` function.

### Programming task: implement `generate_anchors` function.

In [ ]:
# These are the anchor properties that will be used in our implementation.
# Compared to the values picked in the original Faster R-CNN paper, we've
# added two smaller scales that help the model detect the smaller objects
# present in the COCO dataset.
ANCHOR_BASE_SIZE = 256
ANCHOR_RATIOS = [0.5, 1, 2]
ANCHOR_SCALES = [0.125, 0.25, 0.5, 1, 2]


def generate_anchors(feature_map_shape):
    """Generate anchors for an image.

    Using the feature map (the output of the pretrained network for an image)
    and the anchor references (generated using the specified anchor sizes and
    ratios), we generate a list of anchors.

    Anchors are just fixed bounding boxes of different ratios and sizes that
    are uniformly generated throughout the image.

    Arguments:
        feature_map_shape: Shape of the convolutional feature map used as
            input for the RPN.
            Should be (batch, feature_height, feature_width, depth).

    Returns:
        all_anchors: A Tensor with the anchors at every spatial position, of
            shape `(feature_height, feature_width, num_anchors_per_points, 4)`
            using the (x1, y1, x2, y2) convention.
    """

    anchor_reference = generate_anchors_reference(
        ANCHOR_BASE_SIZE, ANCHOR_RATIOS, ANCHOR_SCALES
    )
    
    # Tip: first, implement it with regular Python loops.
    #
    # If you have time and want to try to do it in a vectorized way, the
    # following functions might be of use: `tf.meshgrid`, `tf.range`,
    # `tf.expand_dims` and `tf.transpose`.
    
    ####
    # Fill this function below, paying attention to the docstring.
    ####
    
    ####

    return all_anchors


anchors = tf.reshape(generate_anchors(feature_map.shape), [-1, 4])

print('Anchors (real image size):')
print()
print(anchors.numpy())

Let's draw the anchors over an arbitrary point to corroborate that the results makes sense.

In [ ]:
# Visualize the anchors. Try changing to different points of the image.
# Note that we're referring to _positions in the feature map_ here, so the
# actual point in the image will be around `OUTPUT_STRIDE` times the value.
point = np.array([30, 20])

# Reshape back to (H, W, num_anchors, 4) so we can easily get a given point's anchors.
anchors_at_point = anchors.numpy().reshape(
    (feature_map.shape[1], feature_map.shape[2], 15, 4)
)[point[1], point[0], :, :]

_, ax = plt.subplots(1, figsize=(16, 20))

ax.imshow(to_image(image))
ax.set_xlim([-100, image.shape[2] + 100])
ax.set_ylim([image.shape[1] + 100, -100])

for idx in range(anchors_at_point.shape[0]):
    add_rectangle(ax, anchors_at_point[idx, :])

# Plot the reference point in use.
ax.plot(
    point[0] * OUTPUT_STRIDE,
    point[1] * OUTPUT_STRIDE,
    marker='s', color='#dc3912', markersize=3
)

plt.show()

With this we've finished generating the anchors that will be used by the RPN. This is, effectively, a list of $15 \times F_x \times F_y$, where $F_x, F_y$ are the feature map width and height, respectively.

## Encoding and decoding bounding box coordinates


Deep neural networks usually train and converge better when their outputs have zero mean and unit variance (and/or their intermediate values do so). Due to this, and the difficulty in predicting values in a possibly unbounded region (pixel coordinates), a special encoding is applied to the coordinates before passing them in to the network (and after getting them out).

The idea behind the encoding is to express the coordinates of a bounding box $B$ as a set of four numbers $(D_x, D_y, D_w, D_h)$ (the **deltas**) and a reference anchor $R$. $D_x$ and $D_y$ indicate how much the center of $R$ should be moved to reach the center of $B$, normalized by the size of $R$, while $D_w$ and $D_h$ indicate how much the width and height of $R$ must be increased or decreased to reach the size of $B$ (it's actually the log of that value, as you'll see below).

For the following equations, we change from the $(x_{min}, y_{min}, x_{max}, y_{max})$ encoding to the **center+dimensions encoding** $(x, y, w, h)$, where $(x, y)$ are the **center coordinates**, and $(w, h)$ the **width and height**. The equations to encode $B = (x_b, y_b, w_b, h_b)$ with respect to anchor $R = (x_r, y_r, w_r, h_r)$ are, then:

$D_x = \frac{x_b - x_r}{w_r} \quad$
$D_y = \frac{y_b - y_r}{h_r} \quad$
$D_w = log \frac{w_b}{w_r} \quad$
$D_h = log \frac{h_b}{h_r}$

The equations to decode $B = (x_b, y_b, w_b, h_b)$ given $R = (x_r, y_r, w_r, h_r)$ and deltas $D = (D_x, D_y, D_w, D_h)$ are:

$x_b = D_x w_r + x_r \quad$
$y_b = D_y h_r + y_r \quad$
$w_b = e^{D_w} w_r \quad$
$h_b = e^{D_h} h_r \quad$

We'll implement two functions here, `encode` and `decode`. While only the latter will be used, it's useful to implement both in order to understand the whole process and to make it easier to test.

### Programming task: implement `get_dimensions_and_center` function.

In [ ]:
# You might find it useful to implement the following function first in order
# to obtain the dimensions (width and height) and center of a bounding box,
# required for calculating the deltas in `encode` and `decode`.
def get_dimensions_and_center(bboxes):
    """Obtain width, height and center coordinates of a bounding box.
    
    Arugments:
        bboxes: Tensor of shape (num_bboxes, 4).
        
    Returns:
        Tuple of Tensors of shape (num_bboxes,) with the values
        width, height, center_x and center_y corresponding to each
        bounding box.
    """
    
    # Tip: Fully read the docstring above.
    # Tip: You may find the Tensorflow function `tf.split` useful.

    ####
    # Fill this function below, paying attention to the docstring.
    ####
    
    ####

    return width, height, ctx, cty

### Programming task: implement `encode` function and play around with the checks at the bottom.

In [ ]:
def encode(anchors, bboxes):
    """Encode bounding boxes as deltas w.r.t. anchors.
    
    Arguments:
        anchors: Tensor of shape (num_bboxes, 4). With the same bbox
            encoding.
        bboxes: Tensor of shape (num_bboxes, 4). Having the bbox
            encoding in the (x_min, y_min, x_max, y_max) order.

    Returns:
        Tensor of shape (num_bboxes, 4) with the different deltas needed
            to transform `anchors` to `bboxes`. These deltas are with
            regard to the center, width and height of the two boxes.
    """
   
    ####
    # Fill this function below, paying attention to the docstring.
    ####

    ####

    return deltas


# Encoding `bbox` with respect to an anchor having the same center
# should keep the first two deltas at zero.
anchor = np.array([[0, 0, 100, 100]], dtype=np.float32)
bbox = np.array([[25, 25, 75, 75]], dtype=np.float32)
print('With same center, first two deltas should be zero:\n', encode(anchor, bbox).numpy())
print()

# Encoding `bbox` with respect to an anchor having the same size
# should keep the last two deltas at zero.
anchor = np.array([[0, 0, 100, 100]], dtype=np.float32)
bbox = np.array([[50, 50, 150, 150]], dtype=np.float32)
print('Same size, last two deltas should be zero:\n', encode(anchor, bbox).numpy())

# What other ways to check the functions can you think of?

### Programming task: implement `decode` function, play around with the checks at the bottom.
### Then, verify that that the round trip `encode -> decode` works as expected.

In [ ]:
def decode(anchors, deltas):
    """Decode bounding boxes by applying deltas to anchors.
    
    Arguments:
        anchors: Tensor of shape (num_bboxes, 4). Having the bbox
            encoding in the (x_min, y_min, x_max, y_max) order.
        deltas: Tensor of shape (num_bboxes, 4). Deltas (as returned by
            `encode`) that we want to apply to `bboxes`.

    Returns:
        Tensor of shape (num_bboxes, 4) with the decoded proposals,
            obtained by applying `deltas` to `anchors`.
    """
    
    ####
    # Fill this function below, paying attention to the docstring.
    ####

    ####

    return bboxes


# Decoding `anchor` with zero `deltas` should keep the box as-is.
anchor = np.array([[25, 25, 75, 75]], dtype=np.float32)
delta = np.array([[0, 0, 0, 0]], dtype=np.float32)
print('Zero delta, should get a bounding box with same dimensions:')
print('\tAnchor:', anchor)
print('\tBounding box:', decode(anchor, delta).numpy())
print()

# Applying a `delta` with two ones at first then two zeros to an `anchor`
# should get a bounding box of the size but moved to the right one-width.
anchor = np.array([[25, 25, 75, 75]], dtype=np.float32)
delta = np.array([[1, 1, 0, 0]], dtype=np.float32)
print('First-two are ones, obtained box moved to the right one width:')
print('\tAnchor:', anchor)
print('\tBounding box:', decode(anchor, delta).numpy())
print()

# Decoding `anchor` with two zeros at first then two ones at `deltas`
# should get a larger bounding box while maintaining the center.
anchor = np.array([[25, 25, 75, 75]], dtype=np.float32)
delta = np.array([[0, 0, 1, 1]], dtype=np.float32)
print('Last-two are ones, center should be the same:')
print('\tAnchor:', anchor)
print('\tBounding box:', decode(anchor, delta).numpy())

# What other ways to check the functions can you think of? How can
# you pick the deltas so that it exactly doubles in size?

Let's test the round-trip of `encode` and `decode`, to see if they're consistent between them.

In [ ]:
# Test the round-trip: encode `bboxes` w.r.t. the anchors `anchors`,
# which gives us the deltas that transform `anchors` into `bboxes`.
# Then decode the `anchors` with said deltas to see that, effectively,
# we get `bboxes` back.
anchor = np.array([
    [0, 0, 100, 100],
], dtype=np.float32)

# You can try out other bounding boxes, just make sure to respect the
# convention of first putting (x_min, y_min) then (x_max, y_max), or
# you may get an invalid bounding box.
bboxes = np.array([
    [25, 25, 75, 75],
    [10, -205, 120, 20],
    [-35, 37, 38, 100],
    [-0.2, -0.2, 0.2, 0.2],
    [-25, -50, -5, -20],
], dtype=np.float32)

print(
    'Round-trip looks good:',
    np.sum(np.abs(
        decode(anchor, encode(anchor, bboxes)) - bboxes
    )) < 1e-3
)

If you have time left at the end, you could try to gain further intuition on what they do and what the encoding's edge cases and limitations are by looking at more examples and plotting the deltas as `bboxes` moves through the image.

## Convolutional layers

We now have a variable-size feature map (a factor of 16 times spatially smaller than the original image) and we want to predict, for each spatial position, how to modify (i.e. the $4$ values from above, $D_{x, y, w, h}$) each of the $k = 15$ anchors. In this context, it makes sense to use a convolutional layer (or more) on the feature map, where the final number of filters will be $4 \times k$.

For each of these anchors we'll also want to decide whether we think there's an object present on said region or not (thus, $2 \times k$ more filters). This will, in essence, look at the activation maps we saw before and decide whether, in a given region, the activated features amount to an object being in there (e.g. many _cat face_ features have been activated, so there's probably an object in that region).

As we saw in the slides, the RPN first has a $3\times3$ convolutional layer with $512$ filters and then two outputs heads:
* One with $2 \times k$ filters for the **objectness score**.
* One with $4 \times k$ filters for the **encoded deltas**.

Both will be implemented as $1 \times 1$ convolutions in order to support variable-size images.

### Programming task: implement `run_rpn` function (the forward pass of a RPN).

In [ ]:
# Note that when implementing Faster R-CNN for training, we should
# also specify initializers and regularizers for the weights. We're
# omitting them here for brevity.

def run_rpn(feature_map):
    """Run the RPN layers through the feature map.
    
    Will run the input through an initial convolutional layer of
    filter size 3x3 and 512 channels, using athe ReLU6 activation.
    The output of this layer has the same spatial size as the
    input.
    
    Then run two 1x1 convolutions over this intermediate layer, one
    for the resizings and one for the objectness probabilities.
    Remember to apply the softmax function over the objectness
    scores to get a probability distribution.
    
    Arguments:
        feature_map: Tensor of shape (1, W, H, C), with WxH the
            spatial shape of the feature map and C the number of
            channels (1024 in this case).
            
    Returns:
        Tuple of Tensors, with the first being the output of the bbox
        resizings `(W * H * num_anchors, 4)` while the second being
        the objectness score, of size `(W * H * num_anchors, 2)`.
    """
    
    # Tip: Read the docstring thoroughly to help you pass the correct
    # parameters to the conv layers, especially padding (you want to
    # keep the *same* spatial size after the initial conv layer).
    
    # Tip: See the functions `tf.layers.conv2d` and `tf.reshape`. Also
    # see `tf.nn.softmax` for the softmax function.
    
    # The names of the layers should be: `rpn/conv` for the base layer,
    # `rpn/cls_conv` for the objectness score, and `rpn/bbox_conv` for
    # the bbox resizing.
    
    ####
    # Fill this function below, paying attention to the docstring.
    ####
    
    ####

    return rpn_bbox_pred, rpn_cls_prob


with tfe.restore_variables_on_create('checkpoint/fasterrcnn'):
    rpn_bbox_pred, rpn_cls_prob = run_rpn(feature_map)
    

expected_preds = (
    feature_map.shape[1]
    * feature_map.shape[2]
    * len(ANCHOR_RATIOS)
    * len(ANCHOR_SCALES)
)

assert rpn_bbox_pred.shape[0] == expected_preds, 'Number of proposals should match'
assert rpn_cls_prob.shape[0] == expected_preds, 'Number of proposals should match'

assert rpn_bbox_pred.shape[1] == 4, 'There should be one delta per bbox coordinate (i.e., four)'
assert rpn_cls_prob.shape[1] == 2, 'The objectness score should have two outputs'

## Generating proposals from the RPN output

We now have the RPN layers outputs as-is. These will be the basis for *regions of interest* that will go through to the next stage of the object detection pipeline.

Remember that the RPN layers outputs are the **encoded deltas**. So we need to get them back to image pixel space. When we do this, we can visualize what we have so far!

First we decode the outputs of the RPN using our previously-implemented `decode` function, obtaining **proposals**. We also get a single-dimension **objectness score** for each of these proposals.

In [ ]:
# Generate proposals from the RPN's output by decoding the bounding boxes
# according to the configured anchors.
proposals = decode(anchors, rpn_bbox_pred)

# Get the (positive-object) scores from the RPN.
scores = tf.reshape(rpn_cls_prob[:, 1], [-1])

Keep in mind that we will have **more than 22k proposals** as output, and most will actually be garbage.

In order to visualize what we have so far, we need to first sort them by score, and only keep those with the highest score.

### Programming task: implement `keep_top_n` function.

In [ ]:
def keep_top_n(proposals, scores, topn):
    """Keeps only the top `topn` proposals, ordered by score.
    
    Arguments:
        proposals: Tensor of shape (num_proposals, 4), holding the
            coordinates of the proposals' bounding boxes.
        scores: Tensor of shape (num_proposals,), holding the
            scores associated to each bounding box.
        topn (int): Number of proposals to keep.
        
    Returns:
        (`min(num_proposals, topn)`, `scores`) ordered by score.
    """

    # Tip: See `tf.minimum`, `tf.nn.top_k` to get the top values, and
    # `tf.gather` to select indices out of a Tensor.
    
    ####
    # Fill this function below, paying attention to the docstring.
    ####
    
    ####
    
    return sorted_top_proposals, sorted_top_scores

### Learn: play around with displaying a different number of proposals

Then, answer the following questions:

1. What do you see?
2. Why does it happen? Does it make sense?
3. What problem or problems do we have?
4. What would happen if we had initialized the network with random weights instead of using a pre-trained checkpoint?
5. How could we fix these issues?

In [ ]:
topn = 3000

top_raw_proposals, top_raw_scores = keep_top_n(proposals, scores, topn)
# Display the first `topn` proposals, as ordered by score.
@interact(
    topn=pager(500, 1, min=1, value=10, description='Number of proposals')
)
def draw(topn):
    print('Minimum score: {:.2f}'.format(top_raw_scores[topn]))
    return draw_bboxes(image, top_raw_proposals[:topn])

### Learn: making sense of the RPN deltas

Let's plot a histogram of the bounding box modifications (the deltas) for our current image.

Look at the results. Do they make sense? Does it seem that the encoding is indeed helping unbias the predictions? What do values near zero mean?

In [ ]:
preds = rpn_bbox_pred.numpy()

_, axes = plt.subplots(2, 2, figsize=(16, 12))
for idx, ax in enumerate(axes.ravel()):
    title = ['D_x', 'D_y', 'D_w', 'D_h'][idx]
    ax.set_title(title)
    ax.hist(preds[:, idx], bins=50)
    
plt.show()

Let's also plot the objectness scores. As you'll see, most of the anchors are deemed not worthy.

In [ ]:
preds = rpn_cls_prob.numpy()[:, 1]

_, ax = plt.subplots(1, figsize=(16, 6))
ax.set_title('Scores (0 = no object, 1 = object)')
ax.hist(preds, bins=100)

print('{} predictions over 0.9, out of a total of {}'.format(
    len(np.flatnonzero(preds > 0.9)), len(preds)
))
print()
    
plt.show()

If you have some time left, it may prove insightful to analyze other statistics, such as the objectness and/or resizing by anchor size, or by position in the image. Performing an analysis like this can help pick hyperparameters, guide improvements for the algorithms and find pathologies on the architecture.

## Filtering proposals

As we saw above, it would be smart to implement a stage where we **filter** the proposals that we have, in order to perform better object detection in a later phase.

* Some of the proposals may end up being invalid, as no constraints have been placed on the resizings (aside from the regularization induced by the encoding). For instance, we may end up with **zero-area proposals**, or with the extremes flipped. This may be especially true when we're training the algorithm from scratch (with randomly initialized weights), but we're going to filter them just in case.
* Many of the proposals may end up being **very similar to each other**. Due to this, we're going to apply an operation called **non-maximum suppression** to keep only those proposals that are most different to each other, and enable lower-score but more diverse proposals to get into our final set (to improve the quality of our detections).


First, we will plot the area per proposal in order to visualize how it is distributed, and see if we have some proposals with zero or negative area (in this case, negative area means that the bounding box extremes were flipped). As we said before, it is very much possible that since we're using fully-trained weights, no proposals with negative area are present. You might want to see the `encode` and `decode` functions you implemented above to see exactly when it can go negative.

In [ ]:
props = proposals.numpy()
areas = (props[:, 2] - props[:, 0]) * (props[:, 3] - props[:, 1])

_, ax = plt.subplots(1, figsize=(16, 6))
ax.set_title('Area per proposal')
ax.hist(areas, bins=100)

plt.show()

print('Proposals with areas under zero:')
print(np.flatnonzero(areas <= 0))

### [Optional] Programming task: implement `filter_proposals` function.

In [ ]:
# N.B.: You might as well skip this step if you're running out of time and
# there are no proposals with area under zero, but beware that on a real
# implementation ignoring this will cause trouble.

def filter_proposals(proposals, scores):
    """Filters non-positive area proposals.
    
    Arguments:
        proposals: Tensor of shape (num_proposals, 4), holding the
            coordinates of the proposals' bounding boxes.
        scores: Tensor of shape (num_proposals,), holding the
            scores associated to each bounding box.
        
    Returns:
        (`proposals`, `scores`), but with non-positive area proposals removed.
    """
    
    # Tip: see `tf.greater`, `tf.maximum`, `tf.boolean_mask`.
    
    ####
    # Fill this function below, paying attention to the docstring.
    ####
    
    ####

    return proposals, scores


# Filter proposals with negative areas.
proposals, scores = filter_proposals(proposals, scores)

## Removing redundancy: non-maximum supression

Now we're going to use non-maximum suppression on the list of proposals we have. The end result will be a reduced list of proposals (in fact, of size `POST_NMS_TOP_N` defined below), ordered by objectness score, with some redundancy removed (that is, proposals that are too similar to each other will be discarded).

As explained in [1], NMS greedily selects a subset of bounding boxes in descending order of score, pruning away boxes that have high intersection-over-union (IOU) [2] overlap with previously selected boxes.

We'll be using `NMS_THRESHOLD` as the **IOU overlap threshold**. Also, in order to speed up the NMS (as we may have tens of thousands of proposals, depending on the image size), we'll first limit our proposal list to the top `PRE_NMS_TOP_N` proposals ordered by score.

We'll use an already-implemented Tensorflow function for NMS itself. While this avoids the need to code the algorithm, we need to prepare the parameters correctly to feed it.

You can read more about non-maximum suppression [here](https://www.pyimagesearch.com/2014/11/17/non-maximum-suppression-object-detection-python/) and [here](https://www.pyimagesearch.com/2015/02/16/faster-non-maximum-suppression-python/).

* [1] https://www.tensorflow.org/api_docs/python/tf/image/non_max_suppression
* [2] https://en.wikipedia.org/wiki/Jaccard_index

In [ ]:
# Limit of the initial proposal list, to reduce the number of proposals fed to
# non-maximum suppression.
PRE_NMS_TOP_N = 12000

# We will use the `keep_top_n` function that you have implemented before!
proposals, scores = keep_top_n(proposals, scores, PRE_NMS_TOP_N)

With the proposals pre-filtered, let's now apply NMS.

### Programming task: implement `apply_nms` function, together with the `change_order` helper function which will be useful for you.

In [ ]:
# Final maximum number of proposals, as returned by NMS.
POST_NMS_TOP_N = 2000

# IOU overlap threshold for the NMS procedure.
NMS_THRESHOLD = 0.7


# You might find the following function useful for re-ordening the coordinates
# as expected by Tensorflow.
def change_order(bboxes):
    """Change bounding box encoding order.

    Tensorflow works with the (y_min, x_min, y_max, x_max) order while we work
    with the (x_min, y_min, x_max, y_min).

    While both encoding options have its advantages and disadvantages we
    decided to use the (x_min, y_min, x_max, y_min), forcing us to switch to
    Tensorflow's every time we want to use function that handles bounding
    boxes.

    Arguments:
        bboxes: A Tensor of shape (total_bboxes, 4).

    Returns:
        bboxes: A Tensor of shape (total_bboxes, 4) with the order swaped.
    """
    
    # Tip: see `tf.unstack`, `tf.stack`.

    ####
    # Fill this function below, paying attention to the docstring.
    ####
    
    ####
    
    return bboxes


def apply_nms(proposals, scores):
    """Applies non-maximum suppression to proposals.
    
    Arguments:
        proposals: Tensor of shape (num_proposals, 4), holding the
            coordinates of the proposals' bounding boxes.
        scores: Tensor of shape (num_proposals,), holding the
            scores associated to each bounding box.
        
    Returns:
        (`proposals`, `scores`), but with NMS applied, and ordered by score.
    """
    
    # Tip: See `tf.image.non_max_suppression` to perform NMS, our `change_order`
    # to prepare the bounding boxes, and `tf.gather` to pick indices out of a
    # Tensor.
    
    ####
    # Fill this function below, paying attention to the docstring.
    ####

    ####

    return proposals, scores

pre_merge_proposals, pre_merge_scores = proposals, scores
proposals, scores = apply_nms(proposals, scores)

## Learn: what have we detected? Play around with NMS.

Let's take a look at our current results, so we can understand what we have to work with.

1. How is it different than before?
2. Is the importance of something like NMS more clear now?

In [ ]:
# Display the first `topn` proposals, as ordered by score.
@interact(
    nms=Checkbox(value=True, description='Apply NMS'),
    topn=pager(200, 1, min=1, value=10, description='Number of proposals')
)
def draw(nms, topn):
    if nms:
        p = proposals
        s = scores
    else:
        p = pre_merge_proposals
        s = pre_merge_scores
        
    print('Minimum score: {:.2f}'.format(s[topn]))
    return draw_bboxes(image, p[:topn])

Let's check how the center positions have changed pre- and post- merging of proposals when restricted to the first $2000$ proposals. After applying NMS, we should have improved our coverage of the image somewhat.

In [ ]:
top_k = tf.nn.top_k(pre_merge_scores, k=proposals.shape[0])
props = tf.gather(pre_merge_proposals, top_k.indices).numpy()

pre_merge_centers = np.stack([
    (props[:, 0] + props[:, 2]) / 2,
    (props[:, 1] + props[:, 3]) / 2,
], axis=1)

post_merge_centers = np.stack([
    (proposals[:, 0] + proposals[:, 2]) / 2,
    (proposals[:, 1] + proposals[:, 3]) / 2,
], axis=1)

_, axes = plt.subplots(2, 2, figsize=(16, 8))
axes[0][0].set_title('x-axis center positions pre-merge')
axes[0][0].hist(pre_merge_centers[:, 0], bins=40)
axes[0][1].set_title('y-axis center positions pre-merge')
axes[0][1].hist(pre_merge_centers[:, 1], bins=40)
axes[1][0].set_title('x-axis center positions post-merge')
axes[1][0].hist(post_merge_centers[:, 0], bins=40)
axes[1][1].set_title('y-axis center positions post-merge')
axes[1][1].hist(post_merge_centers[:, 1], bins=40)

plt.show()

## RPN Conclusions

This concludes the work on the Region Proposal Network! We now have a mechanism to, given an image of arbitrary size, return **regions of interest** (i.e. proposals), where it looks like an object is present.

Having gone through all the steps, from generating anchors around the image to predicting and filtering proposals, we now have a list of `POST_NMS_TOP_N` proposals (two thousand, in this case), each with an objectness score assigned.

Two thousand proposals are, of course, many more than what we need. Also, we need to assign an actual class to each of these proposals, or discard them if they're not correct. That will be attacked by the rest of our object detection pipeline.

---
# Standardizing proposals: Region of Interest Pooling

So far, we have obtained regions of interest for an arbitrarily-sized input image. Thousands of them. And all of them of a different size. As you've probably seen in the last visualization, some of them may be very small while others very big.

The objective of this stage is twofold:
1. Get the proposals, defined in **pixel-space coordinates**, back to the **feature maps**.
2. Get them all into a **fixed size** so they can later be fed into a fully-connected neural network.

This final size of each region of interest will be $7\times7\times1024$.
> * $1024$ is the number of filters that our feature map has.
> * $7\times7$ corresponds to the common spatial size all proposals will have.
>
> This implies, as you might notice, that the aspect ratio of the proposals will change.

On the original Faster R-CNN paper, a technique called RoI pooling is used. Here, instead, we use the `tf.image.crop_and_resize` Tensorflow function, which is (in performance terms) almost equivalent but simpler to implement.

Also, bear in mind that the RoI pooling layer **first** resizes to *double* of the pooling size (i.e. gets regions of $14\times14$) and then uses max pooling to get the final $7\times7$ regions. This makes the resulting regions more smooth and makes them capture more details. One could even go further and resize to $28\times28$ or more, but since we're **making a copy** of the feature map, memory usage will rapidly go up (trade-offs...).

So much for an introduction. The implementation should be relatively straightforward. So go on ahead!

### Programming task: implement `roi_pooling` function and the `normalize_bboxes` helper.

In [ ]:
def normalize_bboxes(proposals, im_shape):
    """
    Gets normalized coordinates for RoIs (between 0 and 1 for cropping)
    in TensorFlow's order (y1, x1, y2, x2).

    Arguments:
        roi_proposals: A Tensor with the bounding boxes of shape
            (total_proposals, 4), where the values for each proposal are
            (x_min, y_min, x_max, y_max).
        im_shape: A Tensor with the shape of the image (height, width).

    Returns:
        bboxes: A Tensor with normalized bounding boxes in TensorFlow's
            format order. Its should is (total_proposals, 4).
    """
    
    # See `tf.unstack`, `tf.stack`, `tf.cast`.
    
    ####
    # Fill this function below, paying attention to the docstring.
    ####

    ####

    return bboxes



def roi_pooling(feature_map, proposals, im_shape, pool_size=7):
    """Perform RoI pooling.

    This is a simplified method than what's done in the paper that obtains
    similar results. We crop the proposal over the feature map and resize it
    bilinearly.
    
    This function first resizes to *double* of `pool_size` (i.e. gets
    regions of (pool_size * 2, pool_size * 2)) and then uses max pooling to
    get the final `(pool_size, pool_size)` regions.
    
    Arguments:
        feature_map: Tensor of shape (1, W, H, C), with WxH the spatial
            shape of the feature map and C the number of channels (1024
            in this case).
        proposals: Tensor of shape (total_proposals, 4), holding the proposals
            to perform RoI pooling on.
        im_shape: A Tensor with the shape of the image (height, width).
        pool_size (int): Final width/height of the pooled region.
    
    Returns:
        Pooled feature map, with shape `(num_proposals, pool_size, pool_size,
        feature_map_channels)`.
    """
    
    # Tip: See `tf.image.crop_and_resize` to get crops out of the feature map
    # and resize them.
    
    # Tip: You can **ignore the `box_ind` argument** by passing an array of the
    # correct size filled with zeros (one per proposal). This is because we are
    # using batch size of one.
    
    # Tip: Remember to resize to `2 * pool_size` first.
    
    # Tip: Remember to perform the max pooling as described above, by using
    # the `tf.nn.max_pool` function.
    
    # N.B.: You can resize to `(pool_size, pool_size)` directly and avoid the
    # max pooling step, though the results *will* be inferior.
    
    ####
    # Fill this function below, paying attention to the docstring.
    ####

    ####

    return pooled


pooled = roi_pooling(feature_map, proposals, (image.shape[1], image.shape[2]))

In order to gain an intuition on what exactly is being done here, let's now visualize our **pooled regions of interest**, along with the image patches they come from.

In [ ]:
pool = pooled.numpy()

# Pool the images too to visualize, but using a higher pooling size so
# we don't lose too much resolution.
image_crops = roi_pooling(
    image, proposals,
    (image.shape[1], image.shape[2]),
    pool_size=140
).numpy().astype(np.uint8)


@interact(
    fm_idx=pager(pool.shape[-1], 1, 'Feature map index'),
    im_idx=pager(pool.shape[0], 25, 'Proposals')
)
def display_pooled_proposal(fm_idx=0, im_idx=0):
    axes = image_grid(25, 5, sizes=(3, 3))
    
    for idx, ax in enumerate(axes):
        if im_idx * 25 + idx >= pool.shape[0]:
            break
            
        fm = (
            pool[idx, :, :, fm_idx]
            / pool[idx, :, :, fm_idx].max()
            * 255
        ).astype(np.uint8)
        
        # Get the pooled image regions.
        img = image_crops[im_idx * 25 + idx, ...]
        
        fm_image = Image.fromarray(fm, mode='L').convert('RGBA')
        fm_image = fm_image.resize(img.shape[0:2][::-1], resample=Image.NEAREST)

        # Add some alpha to overlay it over the image.
        fm_image.putalpha(120)

        base_image = Image.fromarray(img)
        base_image.paste(fm_image, (0, 0), fm_image)
        
        ax.imshow(base_image, aspect='auto')

    plt.subplots_adjust(wspace=.02, hspace=.02)
    plt.show()

---
# Using the proposals: Region-CNN

We're ready for the final stage! Here we'll be doing two things:
* Running our set of fixed-sized proposals through a network akin to what was done in the RPN: one input, two outputs. In this case, instead of an objectness score, the output will be a class score (plus a possible **background** score).
* Get these thousands of proposals into a reasonable number. We'll be performing NMS again, but this time per class.

As you can see, it looks like more of the same, which (save some details) it effectively is.

In [ ]:
# We're finally ready to perform the classification, so load the class names.
with open('checkpoint/classes.json') as f:
    classes = json.load(f)
    
print(classes)

## The classification network

As we mentioned before, this last stage will get the proposals through a fully-connected layer. However, before doing that, we'll perform a bit more feature extraction.

You might remember when you were implementing the RPN that we used the first three out of four blocks of the ResNet for feature extraction, discarding the final block. The reasoning behind this move was to leverage the fact that the block three should, in principle, detect more *abstract features* than the final, block four. Now we're ready to perform the final classification, so we will first pass our proposals (which are cuts, although resized, of the original feature map) through **the block four of the ResNet**.

Also, once we do this, since we've already extracted all the features we care about, we'll perform **Global Average Pooling** which means, essentialy, to average out the spatial information: we only care that in a given proposal, some feature was present all around. That means we'll be left with a single vector per proposal.

And, finally, pass this fixed-length vector through two fully-connected layers: one for the bounding box resizings and one for the classes. Since we've used the block four of the ResNet already, we'll not be using an intermediate layer.

> ### Dimensions sanity check helper
> In the case of using 2000 proposals and the ResNet as we did before:
>
> 1. `proposals` should be of shape $(2000, 7, 7, 1024)$.
> 2. After running through ResNet tail, we should get something of shape $(2000, 7, 7, 2048)$.
> 3. After Global Average Pooling, we should condense the spatial dimensions and get $(2000, 2048)$.
> 4. Class scores should be $(2000, 81)$ (80 classes + background).
> 5. Box regression scores should be $(2000, 320)$ ($80 \times 4$ since we have 4 coordinates for each box).


In [ ]:
print(run_resnet_tail.__doc__)

### Programming task: implement `run_rcnn` function.

In [ ]:
def run_rcnn(pooled, num_classes):
    """Run the RCNN layers through the pooled features.

    This directly applies a fully-connected layer from `features`
    to the two outputs we want: a class probability (plus the
    background class) and the bounding box resizings (one per
    class).
    
    In order to obtain the class probability, we apply a softmax
    over the scores obtained from the dense layer, similar to the RPN.
    
    Arguments:
        pooled: Pooled feature map, with shape `(num_proposals,
            pool_size, pool_size, feature_map_channels)`.
        num_classes: Number of classes for the R-CNN.
            
    Returns:
        Tuple of Tensors, with the first being the output of the
        bbox resizings `(W * H * proposals, 4)` and the second being
        the class scores, of size `(pool_size ^ 2 * proposals,
        num_classes)`.
    """
    
    # Remember, you need to do three things with `pooled`:
    # * Pass them through the ResNet block four.
    #   (Tip: See the function `run_resnet_tail`s docstring above.)
    # * Perform Global Average Pooling.
    #   (Tip: See the function `tf.reduce_mean`.)
    # * Run them through two fully-connected layers.
    #   (Tip: See the functions `tf.layers.dense`, `tf.nn.softmax`.)
    
    # W.r.t the fully-connected layers, remember:
    # * To add an extra class for the background class.
    # * To have bounding-box resizings **per-class**.
    
    # The names of the layers should be: `rcnn/fc_classifier` for
    # the classification head, and `rcnn/fc_bbox` for the bbox
    # resizing head.
    
    ####
    # Fill this function below, paying attention to the docstring.
    ####

    ####

    return rcnn_bbox, rcnn_cls_prob


with tfe.restore_variables_on_create('checkpoint/fasterrcnn'):
    bbox_pred, cls_prob = run_rcnn(pooled, len(classes))
    
    
assert bbox_pred.shape[0] == pooled.shape[0], 'Number of proposals should match'
assert cls_prob.shape[0] == pooled.shape[0], 'Number of proposals should match'

assert bbox_pred.shape[1] == len(classes) * 4, 'There should be 4 bbox resizings per class'
assert cls_prob.shape[1] == len(classes) + 1, 'There should be 81 class probabilities (remember the background!)'

### Learn: play around with proposals and their corresponding class determined by R-CNN

Let's take a look at the results now, and see whether they make sense.

We'll display the **most probable class for each proposal**, before applying the final class-specific resizing: this is the pooled region of interest, what the classifier actually looked at.

In [ ]:
output_classes = ['background'] + classes
    
preds = np.argmax(cls_prob.numpy(), axis=1)

@interact(page=pager(len(preds), 20, 'Proposals'))
def display_predictions(page):
    axes = image_grid(20, 5, sizes=(3, 3))
    
    for idx, ax in enumerate(axes):
        if 20 * page + idx >= image_crops.shape[0]:
            break
            
        ax.imshow(image_crops[20 * page + idx, ...], aspect='auto')
        ax.set_title(output_classes[preds[20 * page + idx]])

    plt.subplots_adjust(wspace=.02, hspace=.15)
    plt.show()

### Learn: play around with bounding box resizings (object "refinements")

We'll now display the proposals again, but with the **bounding box resizings applied**.

Corrections are done **per-class**. In order to understand how much these predictions vary, we take some proposals and apply the different possible resizings.

For each region, we first display the resizing for the most probable class and then for three other random classes. If the most probable class is background, we ignore it.

Do you notice anything in particular? Which resizing is the one that fits better to the detected object?

In [ ]:
# Target normalization variances to adjust the output of the R-CNN so it trains better.
TARGET_VARIANCES = np.array([0.1, 0.1, 0.2, 0.2], dtype=np.float32)

# We only consider proposals for which the most-probable class was non-background.
preds = np.argmax(cls_prob.numpy(), axis=1)
non_background = (preds != 0)

non_bg_preds = preds[non_background]
non_bg_proposals = proposals.numpy()[non_background]
non_bg_bboxes = bbox_pred.numpy()[non_background]
non_bg_count = len(np.flatnonzero(non_background))


@interact(page=pager(non_bg_count, 3, 'Proposals'))
def display_resizings(page):
    _, axes = plt.subplots(3, 5, figsize=(16, 10))
    
    for row_idx, cols in enumerate(axes):
        for col in cols:
            col.axis('off')

        proposal_idx = 3 * page + row_idx    
        if proposal_idx >= non_bg_count:
            continue
        
        # Original region.
        # (Using original region size so comparison is easier to the eye.)
        x_min, y_min, x_max, y_max = clip_boxes(
            non_bg_proposals[proposal_idx:proposal_idx + 1],
            image.shape[1:3]
        )[0].numpy().astype(np.int)
        cols[0].imshow(image[0, y_min:y_max, x_min:x_max, :])
        cols[0].set_title('Region')
        
        # Per-class region, correct class first.
        class_ids = np.concatenate([
            np.array([non_bg_preds[proposal_idx] - 1]),
            np.random.randint(0, len(classes), 3)
        ])
        for col, class_id in zip(cols[1:], class_ids):
            cls_bbox_pred = non_bg_bboxes[
                proposal_idx:proposal_idx + 1,
                (4 * class_id):(4 * class_id + 4)
            ]

            cls_objects = decode(
                non_bg_proposals[proposal_idx:proposal_idx+1],
                cls_bbox_pred * TARGET_VARIANCES
            ).numpy()
            
            x_min, y_min, x_max, y_max = clip_boxes(
                cls_objects, image.shape[1:3]
            )[0].numpy().astype(np.int)

            col.imshow(image[0, y_min:y_max, x_min:x_max, :])
            col.set_title(classes[class_id])
        
    plt.subplots_adjust(wspace=.02, hspace=.15)
    plt.show()

## Filtering the object candidates

We're finally getting there! We have one last step to do: getting the final predictions.

What we have now is a list of `POST_NMS_TOP_N` proposals (around $2000$), each with 81 class scores (80 classes plus the background) and 80 bounding box resizings. Out of this, we'll have a total of $2000 \times 80 = 160000$ candidate objects, each with a **score** (its class score) and **bounding box resizing**. We do this in order to consider **all** possible object classifications for a given region proposal: if the most-probable class of a bounding box has a score of $0.48$ and the second one has a score of $0.47$, it is important to consider **both** variants, and not just the highest-scored one.

Of course, we'll not really build the $160000$ proposals at once, but instead perform NMS **on a class-by-class basis**, keeping only the top $100$ proposals per class. Then we'll order all the proposals and keep only the top $300$, which will be the output of our algorithm.

**We've already implemented all this part for you**, as it's same as above but on a class-by-class basis. With a little work, you should be able to do it by yourself if you want.

In [ ]:
objects, labels, probs = rcnn_proposals(
    proposals, bbox_pred, cls_prob, image.shape[1:3], 80,
    min_prob_threshold=0.0,
)

objects = objects.numpy()
labels = labels.numpy()
probs = probs.numpy()

In [ ]:
print('Number of detections above 0.1 probability:', len(labels[probs > 0.1]))
print('Number of detections above 0.5 probability:', len(labels[probs > 0.5]))
print('Number of detections above 0.7 probability:', len(labels[probs > 0.7]))
print()

# Accumluated probability score per class.
probs_per_class = np.bincount(labels, weights=probs, minlength=len(classes))
top_n = probs_per_class.argsort()[::-1][:5]
print('Top 5 predicted classes:')
for cls_idx in top_n:
    print('   {}: {} ({:.2f})'.format(cls_idx, classes[cls_idx], probs_per_class[cls_idx]))
    
_, ax = plt.subplots(1, figsize=(16, 4))
ax.bar(np.arange(80), probs_per_class)

plt.show()

At the end of the day, however, we don't want $300$ low-quality detections, but whatever is good. What we do in practice is to filter detections by their probability score (which is the candidate object's class score).

Let's take a look at the $300$ candidate objects and see how much the predictions change when filtering by score. In the real world, you'll probably use a threshold above $0.5$, depending on your desired precision vs. recall trade-off (too high a threshold will miss detections, while too low will add noise).

In [ ]:
slider = FloatSlider(
    min=0.0, max=1.0, step=0.01, value=0.7,
    description='Probability threshold',
    layout=Layout(width='600px'),
    style={'description_width': 'initial'},
    continuous_update=False
)

@interact(prob=slider)
def display_objects(prob):
    MAX_TO_DRAW = 50

    mask = probs > prob

    return draw_bboxes_with_labels(
        image, classes,
        objects[mask][:MAX_TO_DRAW],
        labels[mask][:MAX_TO_DRAW],
        probs[mask][:MAX_TO_DRAW],
    )

---
# Summing up

**Congratulations!** You finished your own implementation of Faster R-CNN, one of the state-of-the-art object detection algorithms.

Throughout this notebook you should have learned quite a few things:
* **How modern object detectors work**: what inputs they take, what kinds of operations and logic they do, and how much control we have in their workings.
* In particular, **how Faster R-CNN works**, very much in depth.
* **How to use Tensorflow and numpy in the context of computer vision and object detection**. Going a bit more than the usual "stack three layers and call it a day": we've worked with arbitrarily-sized inputs, used conditionals, filtering and other non-standard functions, all within the Tensorflow graph (meaning it can run entirely within a GPU).
* **How to visualize the inner workings of an object detection pipeline**. By leveraging an already-trained network, we could see and corroborate each step of the pipeline to understand what goes behind the scenes and whether we made any errors. This process may have also given you some clues in how to improve the algorithm itself.

This, of course, is just the beginning. Some things you could try, going forward, are:
* **Implement the training of a Faster R-CNN model**. We barely touched on this part, using a pre-trained checkpoint provided by us. The training, apart from using the autograd of your favorite deep learning library, requires some extra steps:

    * Implement the **targets**. We're using supervised learning to train this, so how does training data fit into this? We need to train both the RPN and the R-CNN. In order to do this, we need to build mini-batches of training data for both components, by matching ground-truth boxes to our proposals. You can learn how this is done in our implementation of [Luminoth](https://github.com/tryolabs/luminoth).

    * Implement the **loss functions**. Once the targets are in place, we need to select good losses and balance our dataset in order to train correctly. There may be some difficulties when training, as we are optimizing four losses in total (two for the RPN, two for the R-CNN).

* Improve the algorithm itself. Faster-RCNN has been out for a while now, and while it's still very competitive, there are some known improvements to do. For instance, the RPN can be replaced entirely with the **Feature Pyramid Network** (FPN) [1] and the loss exchanged with the **Focal Loss** [2], to obtain the algorithm called **RetinaNet**.


* [1] https://arxiv.org/pdf/1612.03144.pdf
* [2] https://arxiv.org/pdf/1708.02002.pdf